In [1]:
# --------------------------------------
# 📌 STEP 0: Import Important Libraries
# --------------------------------------

In [2]:
import json
from pyais.encode import encode_dict
from collections import defaultdict

In [3]:
# -------------------------------------------------------------
# 📌 STEP 1: FUNCTION TO GENERATE AIS MESSAGES FROM TRACK DATA
# -------------------------------------------------------------

In [4]:
def generate_ais_messages(track):
    """
    Generates AIS messages using pyais from given vessel track data.

    Parameters:
    - track: List of dictionaries containing timestamp, lat, lon, and MMSI.

    Returns:
    - List of AIS messages (dicts) encoded using AIVDM format.
    """
    ais_messages = []

    for point in track:
        lat = point["latitude"]
        lon = point["longitude"]
        timestamp = point["timestamp"]
        mmsi = point["mmsi"]
        speed = point.get("speed", 15.5)
        
        try:
            # Encode using pyais
            encoded_msgs = encode_dict({
                "mmsi": mmsi,
                "lat": lat,
                "lon": lon,
                "speed": speed,       
                "course": 90.0,      # Default course
                "heading": 90,       # Default heading
                "type": 1            # Position report type
            })

            for encoded_msg in encoded_msgs:
                ais_messages.append({
                    "message": "AIVDM",
                    "mmsi": mmsi,
                    "timestamp": timestamp,
                    "payload": encoded_msg
                })

        except Exception as e:
            print(f"❌ Error generating AIS message for MMSI {mmsi} at {timestamp}: {e}")

    return ais_messages

In [5]:
# --------------------------------------------
# 📌 STEP 2: LOAD SIMULATED VESSEL TRACK DATA
# --------------------------------------------

In [6]:
with open("multi_vessel_tracks.json", "r") as f:
    vessel_tracks = json.load(f)

In [7]:
print(vessel_tracks[0])

{'timestamp': '2025-04-30T09:38:40.869824+00:00Z', 'latitude': 30.732393, 'longitude': 121.82739300000003, 'speed': 18, 'mmsi': 123456789}


In [8]:
# -------------------------------------------------
# 📌 STEP 3: GENERATE AIS MESSAGES FOR ALL VESSELS
# -------------------------------------------------

In [9]:
ais_data = generate_ais_messages(vessel_tracks)

In [10]:
# --------------------------------------
# 📌 STEP 4: GROUP AIS MESSAGES BY MMSI
# --------------------------------------

In [11]:
messages_by_mmsi = defaultdict(list)

for msg in ais_data:
    messages_by_mmsi[msg['mmsi']].append(msg)

In [12]:
# ---------------------------------------------
# 📌 STEP 5: DISPLAY SAMPLE MESSAGES PER VESSEL
# ---------------------------------------------

In [13]:
print("\n🔍 Sample AIS messages per vessel:")
for mmsi, msgs in messages_by_mmsi.items():
    print(f"\n🚢 MMSI: {mmsi} — First 5 AIS Messages:")
    for m in msgs[:5]:
        print(m)


🔍 Sample AIS messages per vessel:

🚢 MMSI: 123456789 — First 5 AIS Messages:
{'message': 'AIVDM', 'mmsi': 123456789, 'timestamp': '2025-04-30T09:38:40.869824+00:00Z', 'payload': '!AIVDO,1,1,,A,11mg=5OP2l8ecW`AUM33Q2l1P000,0*69'}
{'message': 'AIVDM', 'mmsi': 123456789, 'timestamp': '2025-04-30T09:43:40.869824+00:00Z', 'payload': '!AIVDO,1,1,,A,11mg=5OP2l8el=DAUUC3Q2l1P000,0*40'}
{'message': 'AIVDM', 'mmsi': 123456789, 'timestamp': '2025-04-30T09:48:40.869824+00:00Z', 'payload': '!AIVDO,1,1,,A,11mg=5OP2l8etk2AUeSCQ2l1P000,0*28'}
{'message': 'AIVDM', 'mmsi': 123456789, 'timestamp': '2025-04-30T09:53:40.869824+00:00Z', 'payload': '!AIVDO,1,1,,A,11mg=5OP2l8f5HfAUmkCQ2l1P000,0*2D'}
{'message': 'AIVDM', 'mmsi': 123456789, 'timestamp': '2025-04-30T09:58:40.869824+00:00Z', 'payload': '!AIVDO,1,1,,A,11mg=5OP2l8f=vLAUv3SQ2l1P000,0*62'}

🚢 MMSI: 987654321 — First 5 AIS Messages:
{'message': 'AIVDM', 'mmsi': 987654321, 'timestamp': '2025-04-30T09:38:41.054395+00:00Z', 'payload': '!AIVDO,1,1,,A,1>e

In [14]:
# -----------------------------------------------
# 📌 STEP 6: SAVE GENERATED AIS MESSAGES TO FILE
# -----------------------------------------------

In [15]:
with open("ais_data.json", "w") as f:
    json.dump(ais_data, f)

print(f"\n🎉 AIS message generation complete! Total messages: {len(ais_data)} saved into 'ais_data.json'")


🎉 AIS message generation complete! Total messages: 11487 saved into 'ais_data.json'
